# Project Pauli operators onto Hilbert subspaces

We show different ways of projecting a weighted linear combination of pauli strings
into a subspace defined by a subset of size $d$ computational basis states. The 
projected operator is stored as a $d \times d$ ``scipy.sparse.coo_matrix``.

As an example we consider the Hamiltonian of the 1D Heisenberg model with periodic
boundary conditions and $L = 22$ spins:
$$
H = \sum_{\langle i, j \rangle}\left( \sigma^x_i\sigma^x_j + \sigma^y_i\sigma^y_j + \sigma^z_i\sigma^z_j \right).
$$

This package provides two tools to perform this projection:

- ``qiskit_addon_sqd.qubit.matrix_elements_from_pauli()``: is a lower-level function
that returns the non-zero matrix elements of a Pauli string and the corresponding indices
of the non-zero elements.

- ``qiskit_addon_sqd.qubit.project_operator_to_subspace()``: is a higher-level function that 
directly returns a ``scipy.sparse`` matrix.

This notebook shows how to use these two tools to produce the same sparse operator.

### Subspace definition

For this example we just generate length-22 random bitstrings. For the projection
functions to work as expected, it is essential that the bitstrings that define
the subspace are unique and sorted in ascending order according to their unsigned
integer representation.

This can be achieved with the ``qiskit_addon_sqd.qubit.sort_and_remove_duplicates()``
function.

In [1]:
import numpy as np
from qiskit_addon_sqd.qubit import sort_and_remove_duplicates

num_spins = 22
rand_seed = 22
np.random.seed(rand_seed)


def random_bitstrings(n_samples, n_qubits):
    return np.round(np.random.rand(n_samples, n_qubits)).astype("int").astype("bool")


bitstring_matrix = random_bitstrings(50_000, num_spins)

# NOTE: It is essential for the projection code to have the bitstrings sorted!
bitstring_matrix = sort_and_remove_duplicates(bitstring_matrix)

print("Subspace dimension: " + str(bitstring_matrix.shape[0]))
print("Full Hilbert space dimension: " + str(2**num_spins))

Subspace dimension: 49718
Full Hilbert space dimension: 4194304


### First method: nonzero matrix elements and indices.

In [2]:
from qiskit.transpiler import CouplingMap
from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian

coupling_map = CouplingMap.from_ring(num_spins)
hamiltonian = generate_xyz_hamiltonian(coupling_map)
print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIXX', 'IIIIIIIIIIIIIIIIIIIIYY', 'IIIIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIIXXII', 'IIIIIIIIIIIIIIIIIIYYII', 'IIIIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIIIIXXIIII', 'IIIIIIIIIIIIIIIIYYIIII', 'IIIIIIIIIIIIIIIIZZIIII', 'IIIIIIIIIIIIIIXXIIIIII', 'IIIIIIIIIIIIIIYYIIIIII', 'IIIIIIIIIIIIIIZZIIIIII', 'IIIIIIIIIIIIXXIIIIIIII', 'IIIIIIIIIIIIYYIIIIIIII', 'IIIIIIIIIIIIZZIIIIIIII', 'IIIIIIIIIIXXIIIIIIIIII', 'IIIIIIIIIIYYIIIIIIIIII', 'IIIIIIIIIIZZIIIIIIIIII', 'IIIIIIIIXXIIIIIIIIIIII', 'IIIIIIIIYYIIIIIIIIIIII', 'IIIIIIIIZZIIIIIIIIIIII', 'IIIIIIXXIIIIIIIIIIIIII', 'IIIIIIYYIIIIIIIIIIIIII', 'IIIIIIZZIIIIIIIIIIIIII', 'IIIIXXIIIIIIIIIIIIIIII', 'IIIIYYIIIIIIIIIIIIIIII', 'IIIIZZIIIIIIIIIIIIIIII', 'IIXXIIIIIIIIIIIIIIIIII', 'IIYYIIIIIIIIIIIIIIIIII', 'IIZZIIIIIIIIIIIIIIIIII', 'XXIIIIIIIIIIIIIIIIIIII', 'YYIIIIIIIIIIIIIIIIIIII', 'ZZIIIIIIIIIIIIIIIIIIII', 'XIIIIIIIIIIIIIIIIIIIIX', 'YIIIIIIIIIIIIIIIIIIIIY', 'ZIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIXXI', 'IIIIIIIIIIIIIIIIIIIYYI

In [3]:
from qiskit_addon_sqd.qubit import matrix_elements_from_pauli
from scipy.sparse import coo_matrix

d = bitstring_matrix.shape[0]

# Initialize the coo_matrix object
operator_from_matrix_elements = coo_matrix((d, d), dtype="complex128")

for pauli in hamiltonian.paulis:
    matrix_elements, row_indices, col_indices = matrix_elements_from_pauli(bitstring_matrix, pauli)
    operator_from_matrix_elements += coo_matrix(
        (matrix_elements, (row_indices, col_indices)), (d, d)
    )

### Higher-level implementation

In [4]:
from qiskit_addon_sqd.qubit import project_operator_to_subspace

operator = project_operator_to_subspace(bitstring_matrix, hamiltonian, verbose=True)

Projecting term 1 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIIIXX ...
Projecting term 2 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIIIYY ...
Projecting term 3 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIIIZZ ...
Projecting term 4 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIXXII ...
Projecting term 5 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIYYII ...
Projecting term 6 out of 66: (1+0j) * IIIIIIIIIIIIIIIIIIZZII ...
Projecting term 7 out of 66: (1+0j) * IIIIIIIIIIIIIIIIXXIIII ...
Projecting term 8 out of 66: (1+0j) * IIIIIIIIIIIIIIIIYYIIII ...
Projecting term 9 out of 66: (1+0j) * IIIIIIIIIIIIIIIIZZIIII ...
Projecting term 10 out of 66: (1+0j) * IIIIIIIIIIIIIIXXIIIIII ...
Projecting term 11 out of 66: (1+0j) * IIIIIIIIIIIIIIYYIIIIII ...
Projecting term 12 out of 66: (1+0j) * IIIIIIIIIIIIIIZZIIIIII ...
Projecting term 13 out of 66: (1+0j) * IIIIIIIIIIIIXXIIIIIIII ...
Projecting term 14 out of 66: (1+0j) * IIIIIIIIIIIIYYIIIIIIII ...
Projecting term 15 out of 66: (1+0j) * IIIIIIIIIIIIZZIIIIIIII ...
Projecting term 16 

### Check that both implementations yield the same coo_matrix

In [5]:
print((operator.power(2) - operator_from_matrix_elements.power(2)).sum())

0j
